# Satellite Location Demo

In [2]:
import micropip
await micropip.install("https://ds-wheels.s3.amazonaws.com/sgp4-2.23-cp312-cp312-pyodide_2024_0_wasm32.whl")

from sgp4.api import accelerated
print(accelerated)

True


Timeliness is an important aspect for countless uses of Earth Observation (EO) data. Humanitarians and emergency response organizations, for example, benefit from imagery immediately before or after an emergency.

Knowing when an image was captured of an area of interested, or when it will be captured next, can help humanitarians emergency response organizations better understand where aid is most needed.

This notebook demonstrates how the location of EO satellites can be computed in an area of interest. This can be used as a proxy of where images were and will be captured. 

These techniques could be combined with those in other notebooks in this repository, such as loading STAC items and calculating how a disaster impacts people, to create automated analysis for past and future disasters.

## Load Libraries and Data

We'll use the [sgp4](https://pypi.org/project/sgp4/) and [skyfield](https://rhodesmill.org/skyfield/) libraries, which help us calculate the position of satellites using complex orbital physics. Shapely will help us with some of the geometry creation and calculations, and Lonboard will be used to visualize the results.

In [3]:
import micropip

In [4]:
deps = [
    "https://ds-wheels.s3.amazonaws.com/arro3_core-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_compute-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/arro3_io-0.3.0-cp312-cp312-emscripten_3_1_58_wasm32.whl",
    "https://ds-wheels.s3.amazonaws.com/geoarrow_rust_core-0.3.0b1-cp38-abi3-emscripten_3_1_58_wasm32.whl",
    "palettable",
    "matplotlib",
    "lonboard==0.10.0b2"
]
await micropip.install(deps)

In [5]:
%pip install skyfield

In [7]:
from skyfield.api import load, wgs84, Timescale

from datetime import datetime, timedelta, timezone

from lonboard import viz, Map, ScatterplotLayer, basemap, PathLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Oranges_9

import geopandas as gpd
import pandas as pd
from shapely import Point, LineString, Polygon

## Calculating the position of a satellite

We'll load information about specific satellites from [Celestrak](https://celestrak.org). Due to the complexities of orbital physics, data about satellites needs to be updated frequently is only accurate for about 1 week before and after "epoch".

For this example, we'll calculate the position of Sentinel 2b.

In [9]:
sentinel_2b_url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=42063"

sentinel_2b = load.tle_file(sentinel_2b_url, reload=True, filename="sentinel_2b")

<class 'OSError'>: cannot download https://celestrak.org/NORAD/elements/gp.php?CATNR=42063 because <urlopen error [Errno 23] Host is unreachable>

In [3]:
satellite = sentinel_2b[0]
satellite

<EarthSatellite SENTINEL-2B catalog #42063 epoch 2024-07-15 04:18:35 UTC>

For sudden-onset emergencies, the time immediately before an after an event are the most critical. We can set a time range of +2 days (48h) from a certain point in time. For the purposes of this notebook, we'll use `now` (the time when the cell is run). These techniques can also be applied in the past, if a before/after analysis is needed.

In [30]:
# Calculate the current UTC time (without microseconds), then creating a time range + 48h

# load timescale
ts = load.timescale()

# load "ephemeris" (https://rhodesmill.org/skyfield/api.html#planetary-ephemerides), which let's us determine if a satellite is illuminated by the sun (as a proxy of if the ground is lit)
eph = load('data/de421.bsp')

now = datetime.now(timezone.utc).replace(microsecond=0)
t1 = now
t2 = now + timedelta(days=2)

To calculate the position of the satellite throughout the time range, we can initiate a timer and then calculate the position at each time step (defined below).

In [5]:
# initiate

timer = t1

df = pd.DataFrame(columns=['satellite', 'timestamp', 'coordinates', 'lng', 'lat', 'daytime'])

rows = []

The `location_iteration` function lodes the geocentric location, calculates the latitude and longitude, converts them to decimal degrees, and saves them as a point coordinate. It also checks if the satellite is sunlit at the time of calculation, which can be used as an approximation of if it is daytime below.

In [6]:
def location_iteration(timer, sat):
    geocentric = sat.at(Timescale.from_datetime(ts, timer))
    lat, lon = wgs84.latlon_of(geocentric)
    longitude = lon.degrees
    latitude = lat.degrees
    daytime = geocentric.is_sunlit(eph)
    coords = Point(longitude, latitude)

    return timer, longitude, latitude, coords, daytime

Then, we can iterate over the time frame. The more frequent the measurements, the slower the calculation takes.

In [7]:
while timer <= t2:
    timer, longitude, latitude, coords, daytime = location_iteration(timer, sentinel_2b[0])

    row = pd.DataFrame({'satellite': satellite.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
    rows.append(row)    

    timer += timedelta(seconds=15) # ~13 seconds

df = pd.concat(rows, ignore_index=True)
df["time_string"] = df["timestamp"].dt.strftime('%Y-%m-%d %X')

We can then save this as a geodataframe.

In [8]:
sentinel_2b_gdf = gpd.GeoDataFrame(df, geometry="coordinates")
sentinel_2b_gdf

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
0,SENTINEL-2B,2024-07-15 08:37:13+00:00,POINT (24.27316 -24.70139),24.273156,-24.701392,True,2024-07-15 08:37:13
1,SENTINEL-2B,2024-07-15 08:37:28+00:00,POINT (24.04854 -25.58559),24.048538,-25.585594,True,2024-07-15 08:37:28
2,SENTINEL-2B,2024-07-15 08:37:43+00:00,POINT (23.82154 -26.46945),23.821536,-26.469454,True,2024-07-15 08:37:43
3,SENTINEL-2B,2024-07-15 08:37:58+00:00,POINT (23.59202 -27.35296),23.592018,-27.352961,True,2024-07-15 08:37:58
4,SENTINEL-2B,2024-07-15 08:38:13+00:00,POINT (23.35985 -28.23610),23.359847,-28.236100,True,2024-07-15 08:38:13
...,...,...,...,...,...,...,...
11516,SENTINEL-2B,2024-07-17 08:36:13+00:00,POINT (-164.49204 56.26238),-164.492042,56.262381,True,2024-07-17 08:36:13
11517,SENTINEL-2B,2024-07-17 08:36:28+00:00,POINT (-164.99363 57.12241),-164.993626,57.122407,True,2024-07-17 08:36:28
11518,SENTINEL-2B,2024-07-17 08:36:43+00:00,POINT (-165.51600 57.98070),-165.515999,57.980698,True,2024-07-17 08:36:43
11519,SENTINEL-2B,2024-07-17 08:36:58+00:00,POINT (-166.06085 58.83712),-166.060850,58.837121,True,2024-07-17 08:36:58


Let's refine our area of interest to see when/where satellites pass overhead. As the 2024 hurricane season in the Americas is under way, we can focus on the US state of Florida for our analysis.

In [31]:
min_lon = x_min = -89
max_lon = x_max = -74
min_lat = y_min = 22
max_lat = y_max = 32

bbox=[min_lon, max_lon, min_lat, max_lat]
bbox

# Defining our AOI as a polygon shape
florida = Polygon([(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max), (x_min,y_min)])

In [32]:
mask_lon = (sentinel_2b_gdf.lng >= min_lon) & (sentinel_2b_gdf.lng <= max_lon)
mask_lat = (sentinel_2b_gdf.lat >= min_lat) & (sentinel_2b_gdf.lat <= max_lat)

aoi = sentinel_2b_gdf.where(mask_lon & mask_lat).dropna()

sentinel_2b_daylit = aoi[aoi["daytime"] == True]
sentinel_2b_daylit

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
1951,SENTINEL-2B,2024-07-15 16:44:58+00:00,POINT (-88.50209 31.20842),-88.502089,31.208424,True,2024-07-15 16:44:58
1952,SENTINEL-2B,2024-07-15 16:45:13+00:00,POINT (-88.74473 30.32453),-88.744734,30.324531,True,2024-07-15 16:45:13
1953,SENTINEL-2B,2024-07-15 16:45:28+00:00,POINT (-88.98418 29.44028),-88.984180,29.440276,True,2024-07-15 16:45:28
7590,SENTINEL-2B,2024-07-16 16:14:43+00:00,POINT (-80.90817 31.36180),-80.908169,31.361801,True,2024-07-16 16:14:43
7591,SENTINEL-2B,2024-07-16 16:14:58+00:00,POINT (-81.15139 30.47798),-81.151385,30.477980,True,2024-07-16 16:14:58
7592,SENTINEL-2B,2024-07-16 16:15:13+00:00,POINT (-81.39137 29.59380),-81.391373,29.593796,True,2024-07-16 16:15:13
7593,SENTINEL-2B,2024-07-16 16:15:28+00:00,POINT (-81.62830 28.70926),-81.628299,28.709261,True,2024-07-16 16:15:28
7594,SENTINEL-2B,2024-07-16 16:15:43+00:00,POINT (-81.86232 27.82439),-81.862322,27.824393,True,2024-07-16 16:15:43
7595,SENTINEL-2B,2024-07-16 16:15:58+00:00,POINT (-82.09359 26.93920),-82.093591,26.939205,True,2024-07-16 16:15:58
7596,SENTINEL-2B,2024-07-16 16:16:13+00:00,POINT (-82.32225 26.05371),-82.322250,26.053710,True,2024-07-16 16:16:13


Next, we can visualize these points on a map. They are color coded using a continuous color scale, with white being `now` and darker purple being further after `now`.

In [33]:
# This creates a range from 0-1 to define our colormap.
time_norm = (sentinel_2b_daylit.timestamp - t1) / (t2 - t1)

colors = apply_continuous_cmap(time_norm, Oranges_9)

In [34]:
layer = ScatterplotLayer.from_geopandas(
    sentinel_2b_daylit,
    # extensions=[filter_extension],
    get_fill_color=colors,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m = Map(
    layer,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

We can clearly see the passes that Sentinel 2b takes in our area of interest and during our time frame, and during daytime hours.

---

## Calculating the position of a satellite constellation

Usually, humanitarians or emergency response organizations aren't concerned about which satellite captures an image, as long as they have a timely image with the appropriate resolution, (lack of) cloud cover, bands, etc.

The following example calculates the position of Planet satellites. With their portfolio's high spatial and temporal resolution, they are able to capture high-resolution images immediately before or after an event.

In [35]:
planet_url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=planet&FORMAT=tle"

planet_swarm = load.tle_file(planet_url, reload=True, filename="planet_swarm")
planet_swarm[0]

[#################################] 100% planet_swarm


<EarthSatellite SKYSAT-A catalog #39418 epoch 2024-07-15 03:24:19 UTC>

We then calculate the locations for all Planet satellites during our time frame in set increments.

In [14]:
timer = t1

rows = []

while timer <= t2:
    for sat in planet_swarm:
        timer, longitude, latitude, coords, daytime = location_iteration(timer, sat)

        row = pd.DataFrame({'satellite': sat.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
        rows.append(row)    

    timer += timedelta(minutes=2) # ~ 3.5 min

planet_swarm_df = pd.concat(rows, ignore_index=True)
planet_swarm_df["time_string"] = planet_swarm_df["timestamp"].dt.strftime('%Y-%m-%d %X')


In [15]:
planet_swarm_gdf = gpd.GeoDataFrame(planet_swarm_df, geometry="coordinates")

Again, we want to look at the satellites in our AOI. Because Planet satellites capture optical images, the following shows the next passes that occur when the satellite is lit (as a proxy of when the ground below is lit).

In [16]:
mask_lon = (planet_swarm_gdf.lng >= min_lon) & (planet_swarm_gdf.lng <= max_lon)
mask_lat = (planet_swarm_gdf.lat >= min_lat) & (planet_swarm_gdf.lat <= max_lat)

aoi_planet = planet_swarm_gdf.where(mask_lon & mask_lat).dropna()
aoi_planet_day = aoi_planet[aoi_planet.daytime == True]
aoi_planet_day[aoi_planet_day["timestamp"] > now]

planet_day = planet_swarm_gdf[planet_swarm_gdf.daytime == True]
planet_day[planet_day["timestamp"] > now]

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
139,SKYSAT-A,2024-07-15 08:39:13+00:00,POINT (40.68480 74.14524),40.684801,74.145236,True,2024-07-15 08:39:13
140,SKYSAT-B,2024-07-15 08:39:13+00:00,POINT (46.59376 1.94516),46.593755,1.945161,True,2024-07-15 08:39:13
142,SKYSAT-C4,2024-07-15 08:39:13+00:00,POINT (8.36528 49.98751),8.365277,49.987510,True,2024-07-15 08:39:13
144,SKYSAT-C2,2024-07-15 08:39:13+00:00,POINT (163.12304 65.38005),163.123040,65.380053,True,2024-07-15 08:39:13
145,SKYSAT-C3,2024-07-15 08:39:13+00:00,POINT (-20.54920 -75.04930),-20.549196,-75.049297,True,2024-07-15 08:39:13
...,...,...,...,...,...,...,...
200291,FLOCK 4Q-7,2024-07-17 08:37:13+00:00,POINT (-36.61726 -81.85808),-36.617260,-81.858078,True,2024-07-17 08:37:13
200292,FLOCK 4Q-15,2024-07-17 08:37:13+00:00,POINT (13.93084 -63.44480),13.930838,-63.444799,True,2024-07-17 08:37:13
200293,FLOCK 4Q-2,2024-07-17 08:37:13+00:00,POINT (-45.11540 -82.28508),-45.115403,-82.285078,True,2024-07-17 08:37:13
200295,FLOCK 4Q-10,2024-07-17 08:37:13+00:00,POINT (33.92949 33.99620),33.929492,33.996196,True,2024-07-17 08:37:13


## Visualize path of satellites

Instead of visualizing points on a map, a better representation would be a line. We can "connect the dots" calculated in the previous step.

In [20]:
def create_individual_linestrings(coords, timestamps, time_strings):
    coords_list = coords.tolist()
    timestamps_list = timestamps.tolist()
    time_strings_list = time_strings.tolist()
    line_segments = [
        {'linestring': LineString([coords_list[i], coords_list[i + 1]]), 'timestamp': timestamps_list[i], 'time_string': time_strings_list[i]} 
        for i in range(len(coords_list) - 1)
    ]
    return line_segments

In [21]:
new_rows = []

for satellite, group in planet_day.groupby('satellite'):
    line_segments = create_individual_linestrings(group['coordinates'], group['timestamp'], group['time_string'])
    for segment in line_segments:
        new_rows.append({'satellite': satellite, 'linestring': segment['linestring'], 'timestamp': segment['timestamp'], 'time_string': segment['time_string']})

path_segments = gpd.GeoDataFrame(new_rows, geometry='linestring')
path_segments

,satellite,linestring,timestamp,time_string
0,FLOCK 4Q-1,"LINESTRING (29.64148 6.93559, 28.14803 -0.66072)",2024-07-15 08:37:13+00:00,2024-07-15 08:37:13
1,FLOCK 4Q-1,"LINESTRING (28.14803 -0.66072, 26.65205 -8.25277)",2024-07-15 08:39:13+00:00,2024-07-15 08:39:13
2,FLOCK 4Q-1,"LINESTRING (26.65205 -8.25277, 25.11788 -15.83...",2024-07-15 08:41:13+00:00,2024-07-15 08:41:13
3,FLOCK 4Q-1,"LINESTRING (25.11788 -15.83468, 23.50426 -23.4...",2024-07-15 08:43:13+00:00,2024-07-15 08:43:13
4,FLOCK 4Q-1,"LINESTRING (23.50426 -23.40050, 21.75728 -30.9...",2024-07-15 08:45:13+00:00,2024-07-15 08:45:13
...,...,...,...,...
126380,SKYSAT-C9,"LINESTRING (83.57894 18.43737, 82.02691 10.78904)",2024-07-17 08:27:13+00:00,2024-07-17 08:27:13
126381,SKYSAT-C9,"LINESTRING (82.02691 10.78904, 80.52704 3.13462)",2024-07-17 08:29:13+00:00,2024-07-17 08:29:13
126382,SKYSAT-C9,"LINESTRING (80.52704 3.13462, 79.04192 -4.51965)",2024-07-17 08:31:13+00:00,2024-07-17 08:31:13
126383,SKYSAT-C9,"LINESTRING (79.04192 -4.51965, 77.53631 -12.16...",2024-07-17 08:33:13+00:00,2024-07-17 08:33:13


If these line segments are not within the AOI, we can drop them from the dataframe.

In [22]:
def is_within_aoi(linestring, florida):
    return linestring.intersects(florida)

In [23]:
path_segments['within_aoi'] = path_segments['linestring'].apply(lambda x: is_within_aoi(x, florida))
path_segments_clipped = path_segments[path_segments['within_aoi']].drop(columns='within_aoi')

path_segments_clipped['length'] = path_segments_clipped['linestring'].apply(lambda x: x.length)

# This is a way to drop segments that are not proper representations of paths, such as errors caused by traversing the International Date Line, or jumps between calculated points when a satellite enters and exits a nighttime area.
path_segments_clipped = path_segments_clipped[path_segments_clipped["length"] <= 75].drop(columns='length')

As with the single satellite, we can create a color scale that shows the time of the satellite's pass for each line segment. In this example, white is closer to `now` and orange is closer to +48h.

In [41]:
# This creates a range from 0-1 to define our colormap.
time_norm_planet_swarm = (path_segments_clipped.timestamp - t1) / (t2 - t1)

colors_planet_swarm = apply_continuous_cmap(time_norm_planet_swarm, Oranges_9, alpha=.5)

We can roughly estimate the width of the ground coverage of a Planet satellite to be 6km (i.e., the width of an image is 6km). To visualize this, we can set `get_width` to be 6km.

In [42]:
layer2 = PathLayer.from_geopandas(
    path_segments_clipped,
    get_color = colors_planet_swarm,
    get_width=6000,
    opacity=1,
    auto_highlight=True
)

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [43]:
m2 = Map(
    [layer2],
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m2

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

As expected, there are many more passes than by a single satellite. Due to small area of interests for many events, cloud cover, and other variables, a larger constellation lets practitioners then narrow down their search.

The visualization of these paths give practitioners a good sense of the time an image of an area of interest will be captured. For example, a practitioner would be interested in an AOI of this size, but might be particularly interested in images of Miami. They could zoom in to Miami and see if a path covers or comes close to Miami. If so, and if there isn't excessive cloud cover, an image could be made available or tasked. If multiple paths cover Miami, that indicates a better chance of capturing a valuable image.

Once a satellite is calculated to have passed over an AOI, the next step could be to analyze the imagery, ideally using the other STAC and COG focused notebooks in this repository. Combining a positioning calculation with automated image loading and processing could be the foundations for a powerful EO-based monitoring and alerting system.